In [5]:
import numpy as np
import math
t = 0 #time
Th = [] #collector of theta
num_theta = 0 #num theta
A = [] #collector of arm
num_arms = 0 #number of arms
K = 3 #dimension of arm x, y, theta
Y = [] #collector of arm y
num_arms_y = 0 #number of arms of player two
M = [] #world
num_M = 0
M_vero = 0
theta_vero = 0

delta = 0.05
sigma = 0.1 #noise

class world():
    matrix = np.array([])
    id

    def __init__(self, param):
        self.matrix = param
        global M, num_M
        self.id = num_M
        M.append(self)
        num_M +=1

def create_union_arm(id_x, id_y):
    res = []
    for i in range(0, K):
        for j in range(0, K):
            res.append(A[id_x].arm_vector[i] * A[id_y].arm_vector[j])
    return res

class arm():
    arm_vector = np.array([])
    id = 0
    worst_y = -1
    double_arm = []
    reward = [[]]
    
    def __init__(self, vector):
        self.arm_vector = vector
        global num_arms, A, N
        A.append(self)
        self.id = num_arms
        num_arms +=1        
        
    def generate_double(self):
        for i in range(0, num_arms_y):
            double_arm.append(create_union_arm(self.id, i))

    def pull_arm(self, y_index):
        reward = np.dot(self.arm_vector, np.dot(M[M_vero].matrix, np.transpose(Y[y_index].arm_vector))) + np.random.normal(0, +sigma)
        Y[y_index].pull_arm(reward, self.id)
        global N
        N[self.id] += 1
        return reward
    
    def find_worst_y(self):
        print("find worst y")
        mini = 100000000
        index = - 1
        for i in range(0, num_arms_y):
            curr = Y[i].lower_bound(self.arm_vector)
            if curr < mini:
                index = i
                mini = curr
        self.worst_y = index
        #print("mini", mini)
        return mini
    
    def calcola_A(self, y_index):
        res = 0
        for i in range(0,self.times):
            #print("i", i)
            if i == 0:
                #print("i==0, print res", res)
                #print(" A[self.A_id[i]].arm_vector",  A[self.A_id[i]].arm_vector)
                res = np.outer(np.transpose(A[self.A_id[i]].double_arm[y_index]), A[self.A_id[i]].double_arm[y_index])
                #print("res", res)
            else:
                #print("i!=0, print i, res", i, res)
                res += np.outer(np.transpose(A[self.A_id[i]].double_arm[y_index]), A[self.A_id[i]].double_arm[y_index])
        #print("A", res)
        return res

    def calcola_b(self, y_index):
        res = []
        global A
        for i in range(0,self.times):
            curr = A[self.A_id[i]].double_arm[y_index]
            curr = [self.R[i] * xx for xx in curr]
            if i == 0:
                res = curr
            else:
                [x + y for x, y in zip(res, curr)]
        return res
        
    def stima_theta(self):
        if (self.times == 0):
            return self.theta_stimato
        theta_s = np.dot(np.linalg.inv(self.calcola_A()), self.calcola_b())
        self.theta_stimato = theta_s
        return theta_s
    
        
    def norm(self):
        return np.linalg.norm(self.arm_vector)
    
class arm_y():
    arm_vector = np.array([])
    id = 0
    best_theta = -1

    def __init__(self, vector):
        self.arm_vector = vector
        global num_arms_y, Y
        Y.append(self)
        self.id = num_arms_y
        num_arms_y +=1

In [6]:
a = arm([0.1, 0.3, 0.6])
b = arm([0.6, 0.2, 0.5])
c = arm([0.4, 0.4, 0.2])

f = arm_y([0.8, 0.3, 0.4])
g = arm_y([0.6, 0.5, 0.5])
h = arm_y([0.1, 0.2, 0.7])


In [7]:
create_union_arm(0,0)

[0.010000000000000002, 0.03, 0.06, 0.03, 0.09, 0.18, 0.06, 0.18, 0.36]